In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler
from matplotlib import pyplot as plt

#### Load data

In [2]:
confirmed_path = "../covid19_confirmed_global.txt"
death_path = "../covid19_deaths_global.txt"

In [3]:
confirmed = pd.read_csv(confirmed_path)
death = pd.read_csv(death_path)
## Select US data
confirmed = confirmed[confirmed['Country/Region'] == 'US']
death = death[death['Country/Region'] == 'US']

## Just Keep Time Series Data
confirmed_train = np.array(confirmed.iloc[:,4::])
death_train = np.array(death.iloc[:,4::])

print(confirmed_train.shape,death_train.shape)

(1, 678) (1, 678)


#### Normalize

In [4]:
#record mean & std
confirmed_mean,confirmed_std = confirmed_train.mean(),confirmed_train.std()
death_mean,death_std = death_train.mean(),death_train.std()

#normalize
confirmed_train = (confirmed_train - confirmed_mean) / confirmed_std 
death_train = (death_train - death_mean) / death_std 

### create seq-to-seq 

In [5]:
def create_dataset(X, time_steps=7):
    Xs, ys = [],[]
    for i in range(len(X) - time_steps*4):
        Xs.append(X[i:(i + time_steps*3)])
        ys.append(X[(i + time_steps*3):(i + time_steps*4)])
    return np.expand_dims(np.array(Xs),axis=2),np.expand_dims(np.array(ys),axis=2)

TIME_STEPS = 7 #a week

#c stands for confimed
X_c, y_c = create_dataset(
  confirmed_train[0],
  TIME_STEPS
)

#d stands for death
X_d, y_d = create_dataset(
  death_train[0],
  TIME_STEPS
)

print(X_c.shape,y_c.shape)
print(X_d.shape,y_d.shape)

(650, 21, 1) (650, 7, 1)
(650, 21, 1) (650, 7, 1)


#### Use LSTM to Predict

In [6]:
def get_model():
    model = keras.Sequential()
    model.add(keras.layers.LSTM(units=256, input_shape=[21,1]))
    model.add(keras.layers.Dense(64))
    model.add(keras.layers.Dense(32))
    model.add(keras.layers.Dense(7))
    return model

#### Confirmed

In [7]:
epochs = 100
batch_size = 32
optimizer = keras.optimizers.Adam(0.0001)

model = get_model()
model.compile(optimizer=optimizer, loss="mse")

history = model.fit(
    X_c, y_c,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=False)

Train on 650 samples
Epoch 1/100
650/650 [==============================] - 2s 3ms/sample - loss: 0.767
Epoch 2/100
650/650 [==============================] - 1s 881us/sample - loss: 0.320
Epoch 3/100
650/650 [==============================] - 1s 916us/sample - loss: 0.078
Epoch 4/100
650/650 [==============================] - 1s 942us/sample - loss: 0.019
Epoch 5/100
650/650 [==============================] - 1s 881us/sample - loss: 0.014
Epoch 6/100
650/650 [==============================] - 1s 931us/sample - loss: 0.014
Epoch 7/100
650/650 [==============================] - 1s 924us/sample - loss: 0.008
Epoch 8/100
650/650 [==============================] - 1s 913us/sample - loss: 0.010
Epoch 9/100
650/650 [==============================] - 1s 926us/sample - loss: 0.011
Epoch 10/100
650/650 [==============================] - 1s 967us/sample - loss: 0.007
Epoch 11/100
650/650 [==============================] - 1s 958us/sample - loss: 0.009
Epoch 12/100
650/650 [======================

In [8]:
last_21 = confirmed_train[0][-21::].reshape(1,21,1)
pred = model.predict(last_21)
pred = (pred * confirmed_std) + confirmed_mean
print("prediction of confirmed in last 7 days:",pred)

tmp = [(confirmed_train[0][-1] * confirmed_std) + confirmed_mean]
tmp.extend(list(pred[0]))
print("The daily new confirmed is:",np.diff(tmp))

prediction of confirmed in last 7 days: [[48503320. 48549190. 48496430. 48679452. 48802000. 48891584. 48916490.]]
The daily new confirmed is: [ 65365.  45872. -52760. 183020. 122548.  89584.  24904.]


#### Death

In [9]:
optimizer = keras.optimizers.Adam(0.0001)
model = get_model()
model.compile(optimizer=optimizer, loss="mse")

history = model.fit(
    X_d, y_d,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=False)

Train on 650 samples
Epoch 1/100
650/650 [==============================] - 2s 4ms/sample - loss: 0.849
Epoch 2/100
650/650 [==============================] - 1s 941us/sample - loss: 0.514
Epoch 3/100
650/650 [==============================] - 1s 974us/sample - loss: 0.092
Epoch 4/100
650/650 [==============================] - 1s 927us/sample - loss: 0.028
Epoch 5/100
650/650 [==============================] - 1s 990us/sample - loss: 0.043
Epoch 6/100
650/650 [==============================] - 1s 887us/sample - loss: 0.017
Epoch 7/100
650/650 [==============================] - 1s 901us/sample - loss: 0.015
Epoch 8/100
650/650 [==============================] - 1s 900us/sample - loss: 0.015
Epoch 9/100
650/650 [==============================] - 1s 906us/sample - loss: 0.010
Epoch 10/100
650/650 [==============================] - 1s 902us/sample - loss: 0.010
Epoch 11/100
650/650 [==============================] - 1s 923us/sample - loss: 0.010
Epoch 12/100
650/650 [======================

In [10]:
last_21 = death_train[0][-21::].reshape(1,21,1)
pred = model.predict(last_21)
pred = (pred * death_std) + death_mean
print("prediction of death in last 7 days:",pred)

tmp = [(death_train[0][-1] * death_std) + death_mean]
tmp.extend(list(pred[0]))
print("The daily new death is:",np.diff(tmp))

prediction of death in last 7 days: [[752528.1 745079.1 749663.4 754008.1 753389.6 748977.4 750531. ]]
The daily new death is: [-26072.875  -7449.      4584.25    4344.75    -618.5    -4412.25
   1553.625]
